In [2]:
import numpy as np
import pandas as pd
import scipy as sp

In [3]:
df_agricultura = pd.read_csv('input/Agricultura.csv')
df_populatie = pd.read_csv('input/PopulatieLocalitati.csv')
df_ronuts = pd.read_csv('input/RO_NUTS.csv')

### 1_LOCALITATILE SORTATE DESCRESCATOR DUPA CIFRA CUMULATA INTRE PLANTE NEPERMANENTE SI PERMANENTE INMULTIRE
- se face un dataframe nou cu structura siruta/localitate/plante (conform cerintei)
- coloana de plante va fi suma dintre coloanele de plante nepermanente si plante inmultire
- se da sort la dataframe dupa coloane plante
- se scrie in csv

In [4]:
df_agricultura_plante = pd.DataFrame()
df_agricultura_plante['Siruta'] = df_agricultura['Siruta']
df_agricultura_plante['Localitate'] = df_agricultura['Localitate']
df_agricultura_plante['Plante'] = df_agricultura['PlanteNepermanente'] + df_agricultura['PlanteInmultire']
df_agricultura_plante.sort_values(by='Plante', ascending=False)
df_agricultura_plante.to_csv('output/cerinta1.csv')

In [5]:
df_agricultura_plante

,Siruta,Localitate,Plante
0,32768,Budesti,0.0
1,49153,Sahateni,0.0
2,57350,Cojocna,0.0
3,90119,Lunca Cernii De Jos,0.0
4,73736,Poiana Mare,0.0
...,...,...,...
3176,122864,Grinties,0.0
3177,163832,Falciu,0.0
3178,40955,Feldioara,0.0
3179,131069,Oras Boldesti-Scaeni,0.0


### 2_SUMA CIFREI DE AFACERI PENTRU FIECARE DINTRE LOCALITATI
- se creaza un dataframe nou cu structura siruta/localitate/cif_afaceri (conform cerintei)
- se extrag intr-un dataframe toate coloanele din agricultura care tin de cif de afaceri (totul de la col 2 la sfarsit)
- coloana cif_afaceri din dataframe-ul nou creat va fi egala cu suma pe linie a dataframe-ului cu valorile ce tin de cif de afaceri
- se scrie in csv

In [6]:
df_agricultura_cif_afaceri = pd.DataFrame()
df_agricultura_cif_afaceri['Siruta'] = df_agricultura['Siruta']
df_agricultura_cif_afaceri['Localitate'] = df_agricultura['Localitate']
#extrage toate liniile de la coloana 2 incolo
df_cif_afaceri = df_agricultura.iloc[:, 2:]
#axis = 1 specifica ca operatie de sum se face la randuri. axis = 0 s-ar fi facut pe coloane
df_agricultura_cif_afaceri['Total_cif_afaceri'] = df_cif_afaceri.sum(axis=1)
df_agricultura_cif_afaceri.to_csv('output/cerinta2.csv')

In [7]:
df_agricultura_cif_afaceri

,Siruta,Localitate,Total_cif_afaceri
0,32768,Budesti,0.0
1,49153,Sahateni,390087.0
2,57350,Cojocna,1670212.0
3,90119,Lunca Cernii De Jos,0.0
4,73736,Poiana Mare,2854738.0
...,...,...,...
3176,122864,Grinties,0.0
3177,163832,Falciu,44473344.0
3178,40955,Feldioara,52518542.0
3179,131069,Oras Boldesti-Scaeni,312983474.0


### 3_CIFRA DE AFACERI PER CAP DE LOCUITOR PENTRU FICARE LOCALITATE IN PARTE
- se creaza un dataframe nou cu structura siruta/localitate/cif_afaceri_per_locuitor (conform cerintei)
- nu am date despre numarul de oameni per localitate asa ca va trebui sa fac merge cu df_populatie
    - fac merge cu df_populatie pentru ca dataframe-urile au o ordine diferita a liniilor
    - pot folosi df_agricultura_cif_afaceri de la punctul trecut ca dataframe de merge
        - mergul pe care il fac este inner pentru ca vreau doar datele care sunt comune ambelor dataframe-urilor

- dupa ce am facut merge, coloana cif_afaceri_per_locuitor din noul dataframe va avea ca valoare raportul dintre nr_loc/total_cif_afaceri 


In [8]:
df_agricultura_cif_afaceri_per_loc = pd.DataFrame()
df_merge_afaceri_pop = df_agricultura_cif_afaceri.merge(df_populatie, how='inner')
df_agricultura_cif_afaceri_per_loc['Siruta'] = df_merge_afaceri_pop['Siruta']
df_agricultura_cif_afaceri_per_loc['Localitate'] = df_merge_afaceri_pop['Localitate']
df_agricultura_cif_afaceri_per_loc['Ca_loc'] = df_merge_afaceri_pop['Total_cif_afaceri']/df_merge_afaceri_pop['Populatie']
df_agricultura_cif_afaceri_per_loc.to_csv('output/cerinta3.csv')

In [9]:
df_agricultura_cif_afaceri_per_loc

,Siruta,Localitate,Ca_loc
0,32768,Budesti,0.000000
1,49153,Sahateni,114.832794
2,57350,Cojocna,398.713774
3,90119,Lunca Cernii De Jos,0.000000
4,73736,Poiana Mare,267.422763
...,...,...,...
3175,122864,Grinties,0.000000
3176,163832,Falciu,7783.224361
3177,40955,Feldioara,7083.698678
3178,131069,Oras Boldesti-Scaeni,26995.297050


### 4_CIFRA DE AFACERI PER SECTOR LA NIVEL DE JUDET
- vom extrage intr-un dataframe judet/activ1/activ2/.../activ_n
    - pentru a face asta extragem toate activitatile si localitatea din df_agricultura 
    - dam merge cu judet din df_populatie
- vom crea un nou dataframe in care vom da groupby la primul dataframe bazat pe judet facand suma pe fiecare linie

In [10]:
df_tip_activ = df_agricultura.iloc[:, 1:]
df_jud_activ_merge = df_tip_activ.merge(df_populatie[['Judet','Localitate']], how='inner')

# daca nu pun as_index = false, nu voi mai putea accesa coloana judet pentru ca devine index
# pentru ca o sa mai folosesc df_cif_afaceri in viitor voi vrea sa am acces la coloana Judet
df_cif_afaceri_jud = df_jud_activ_merge.groupby('Judet', as_index=False).sum()
# (OPTIONAL) nu am nevoie de coloana localitate asa ca ii voi da drop
df_cif_afaceri_jud.drop('Localitate', axis=1)
df_cif_afaceri_jud.to_csv('output/cerinta4.csv')

In [98]:
df_cif_afaceri_jud.sort_values('Judet')

,Judet,Localitate,PlantePermanente,PlanteNepermanente,PlanteInmultire,CrestereaAnimalelor,FermeMixte,ActivitatiAuxiliare
0,ab,Vadu MotilorValea LungaVidraVintu De JosLivezi...,1.007660e+08,157215511.0,2286030.0,675752092.0,17735482.0,29525661.0
1,ag,DobrestiDomnestiDragoslaveleDraganuPopestiGode...,2.346830e+08,2724578.0,199525.0,288616681.0,10678077.0,21267928.0
2,ar,FantaneleMunicipiul AradFantaneleLivadaSofrone...,4.923487e+08,12994009.0,418837.0,111806410.0,41711037.0,35922919.0
3,b,Municipiul Bucuresti,2.516831e+09,15871233.0,13136115.0,216949664.0,49321755.0,69409930.0
4,bc,PoduriRacovaRacaciuniRachitoasaMargineniRosior...,3.125797e+08,24626247.0,2268525.0,108488579.0,239769271.0,169815903.0
5,bh,DobrestiPopestiPopestiRosioriDraganestiPopesti...,6.146658e+08,3385875.0,1546957.0,311095635.0,39514562.0,29000391.0
6,bn,BudestiCaianu MicCetateCiceu-GiurgestiChiochis...,2.362249e+07,17464799.0,758437.0,118183040.0,1809748.0,3963133.0
7,br,RosioriRacovitaUnireaTraianSutestiVictoriaMuni...,1.193549e+09,9157647.0,190712.0,245177247.0,64337325.0,64332160.0
8,bt,RomanestiDraguseniCandestiUngureniVladeniDragu...,3.484512e+08,2178738.0,0.0,69468520.0,58155399.0,22734551.0
9,bv,BunestiFundataHalchiuHarmanHarseniHoghizHomoro...,8.086620e+07,5703015.0,20932953.0,626220718.0,101184584.0,51482320.0


### 5_CIFRA DE AFACERI MEDIE LA NIVEL DE REGIUNE PENTRU FIECARE ACTIVITATE
- nu avem regiunile asa ca va trebui sa facem merge la df_cif_afaceri_jud cu df_ronuts unde si apar
- nu am populatia per regiuni asa ca va trebui sa o calculez pentru ca reprezinta ponderea in formula
- dupa ce am facut merge-urile, similar cu cerinta4, va trebui sa dam un groupby, de aceasta data pe regiuni 
    - trebuie sa folosesc o functie custom pentru media ponderata

prelucrari pentru activitati per regiuni

In [101]:
df_cif_afaceri_regiuni = df_cif_afaceri_jud.merge(df_ronuts[['Regiune','IndicativJudet']], how='inner', left_on='Judet', right_on='IndicativJudet')
#dau delete la indicativ judet pentru ca il folosesc doar pentru merge si nu am nevoie sa apara de doua ori in dataframe
df_cif_afaceri_regiuni.drop(columns=['IndicativJudet'], axis=1, inplace=True)
#nu pot sa calculez media unor valori nenumerice asa ca voi aplica groupby().mean() doar pe coloanele numerice
numeric_columns = df_cif_afaceri_regiuni.select_dtypes('number').columns.to_list()
numeric_columns.append('Regiune')
# df_cif_afaceri_regiuni_grouped = df_cif_afaceri_regiuni[numeric_columns].groupby('Regiune', as_index=False).sum()

In [103]:
df_cif_afaceri_regiuni

,Judet,Localitate,PlantePermanente,PlanteNepermanente,PlanteInmultire,CrestereaAnimalelor,FermeMixte,ActivitatiAuxiliare,Regiune
0,ab,Vadu MotilorValea LungaVidraVintu De JosLivezi...,1.007660e+08,157215511.0,2286030.0,675752092.0,17735482.0,29525661.0,Centru
1,ag,DobrestiDomnestiDragoslaveleDraganuPopestiGode...,2.346830e+08,2724578.0,199525.0,288616681.0,10678077.0,21267928.0,Sud
2,ar,FantaneleMunicipiul AradFantaneleLivadaSofrone...,4.923487e+08,12994009.0,418837.0,111806410.0,41711037.0,35922919.0,Vest
3,b,Municipiul Bucuresti,2.516831e+09,15871233.0,13136115.0,216949664.0,49321755.0,69409930.0,Bucuresti-Ilfov
4,bc,PoduriRacovaRacaciuniRachitoasaMargineniRosior...,3.125797e+08,24626247.0,2268525.0,108488579.0,239769271.0,169815903.0,Nord-Est
5,bh,DobrestiPopestiPopestiRosioriDraganestiPopesti...,6.146658e+08,3385875.0,1546957.0,311095635.0,39514562.0,29000391.0,Nord-Vest
6,bn,BudestiCaianu MicCetateCiceu-GiurgestiChiochis...,2.362249e+07,17464799.0,758437.0,118183040.0,1809748.0,3963133.0,Nord-Vest
7,br,RosioriRacovitaUnireaTraianSutestiVictoriaMuni...,1.193549e+09,9157647.0,190712.0,245177247.0,64337325.0,64332160.0,Sud-Est
8,bt,RomanestiDraguseniCandestiUngureniVladeniDragu...,3.484512e+08,2178738.0,0.0,69468520.0,58155399.0,22734551.0,Nord-Est
9,bv,BunestiFundataHalchiuHarmanHarseniHoghizHomoro...,8.086620e+07,5703015.0,20932953.0,626220718.0,101184584.0,51482320.0,Centru


prelucrari pentru populatie per regiuni

In [106]:
df_pop_per_judet = df_populatie.groupby('Judet', as_index=False).sum()
df_pop_per_judet.drop('Localitate', axis=1, inplace=True)
df_pop_regiuni = df_pop_per_judet.merge(df_ronuts[['Regiune', 'IndicativJudet']], how='inner', left_on='Judet', right_on='IndicativJudet')
df_pop_regiuni.drop('IndicativJudet', axis=1, inplace=True)
df_pop_regiuni.drop('Judet', axis=1, inplace=True)
df_pop_regiuni.drop('Siruta', axis=1, inplace=True)
#df_pop_regiunui_grouped = df_pop_regiuni.groupby('Regiune', as_index=False).sum()

#df_cif_afaceri_regiuni[numeric_columns].groupby("Regiune").mean()
# df_cif_afaceri_regiuni[numeric_columns.extend('Regiune')].groupby(by='Regiune').mean()

In [105]:
df_pop_per_judet

,Judet,Siruta,Populatie
0,ab,401272,381185
1,ag,1689589,646530
2,ar,869159,474347
3,b,179132,2107399
4,bc,2166280,747689
5,bh,2979592,619475
6,bn,2828535,329490
7,br,1920859,356380
8,bt,2962730,456404
9,bv,2397592,631356


In [112]:
df_merge = df_pop_per_judet.merge(df_cif_afaceri_regiuni, on='Judet', how='inner')
for column in df_merge.iloc[:, 4:-1]:
    df_merge[column] *= df_merge['Populatie']/df_merge['Populatie'].sum()

In [113]:
df_merge

,Judet,Siruta,Populatie,Localitate,PlantePermanente,PlanteNepermanente,PlanteInmultire,CrestereaAnimalelor,FermeMixte,ActivitatiAuxiliare,Regiune
0,ab,401272,381185,Vadu MotilorValea LungaVidraVintu De JosLivezi...,1.725477e+06,2.692096e+06,3.914506e+04,1.157131e+07,3.036953e+05,5.055856e+05,Centru
1,ag,1689589,646530,DobrestiDomnestiDragoslaveleDraganuPopestiGode...,6.816000e+06,7.913110e+04,5.794891e+03,8.382419e+06,3.101280e+05,6.176936e+05,Sud
2,ar,869159,474347,FantaneleMunicipiul AradFantaneleLivadaSofrone...,1.049127e+07,2.768845e+05,8.924841e+03,2.382441e+06,8.888049e+05,7.654680e+05,Vest
3,b,179132,2107399,Municipiul Bucuresti,2.382649e+08,1.502508e+06,1.243578e+06,2.053833e+07,4.669222e+06,6.570942e+06,Bucuresti-Ilfov
4,bc,2166280,747689,PoduriRacovaRacaciuniRachitoasaMargineniRosior...,1.049883e+07,8.271390e+05,7.619454e+04,3.643882e+06,8.053298e+06,5.703726e+06,Nord-Est
5,bh,2979592,619475,DobrestiPopestiPopestiRosioriDraganestiPopesti...,1.710496e+07,9.422236e+04,4.304882e+04,8.657190e+06,1.099614e+06,8.070249e+05,Nord-Vest
6,bn,2828535,329490,BudestiCaianu MicCetateCiceu-GiurgestiChiochis...,3.496449e+05,2.585027e+05,1.122590e+04,1.749269e+06,2.678672e+04,5.865974e+04,Nord-Vest
7,br,1920859,356380,RosioriRacovitaUnireaTraianSutestiVictoriaMuni...,1.910789e+07,1.466076e+05,3.053167e+03,3.925118e+06,1.029996e+06,1.029913e+06,Sud-Est
8,bt,2962730,456404,RomanestiDraguseniCandestiUngureniVladeniDragu...,7.144152e+06,4.466977e+04,0.000000e+00,1.424285e+06,1.192336e+06,4.661172e+05,Nord-Est
9,bv,2397592,631356,BunestiFundataHalchiuHarmanHarseniHoghizHomoro...,2.293510e+06,1.617477e+05,5.936959e+05,1.776074e+07,2.869776e+06,1.460131e+06,Centru
